### Imports

In [ ]:
%pip install -r dependencies.txt

In [3]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)

<module 'helper' from '/Users/mike/Documents/School/Fa23/Clinic/code/helper.py'>

### CONSTANTS

In [4]:
ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

In [5]:
REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
REGRESSION_TYPES

1: OLS
2: LASSO
3: XGBOOST


### Read data

In [6]:
train_start, train_end  = helper.get_train_from_testday(TEST_START_DATE)
train_start, train_end

('20140501', '20150531')

In [7]:
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0", "relvol_nt_0"]
train_df = helper.get_df(TRAIN_START_DATE, TRAIN_END_DATE, x_cols)

In [8]:
train_df.info

<bound method DataFrame.info of       rrirpnxm_nt_0  rrirpnxm_lst15_0  rrirpnxm_lsthrx15_0  rrirpnxm_toxhr_0  \
0          0.000000          0.000000             0.000000          0.000000   
1         -0.003448          0.001921             0.001569         -0.013866   
2          0.000000         -0.002830            -0.001450          0.004245   
3          0.000000          0.000000             0.000000          0.000000   
4         -0.001338          0.005082            -0.000502         -0.010742   
...             ...               ...                  ...               ...   
3141      -0.021104         -0.002963            -0.008623          0.004042   
3142      -0.002111         -0.002174            -0.002763          0.011052   
3143       0.004007         -0.002899            -0.003872          0.006881   
3144       0.000518         -0.003926            -0.002201          0.000419   
3145       0.000000         -0.001642             0.001583          0.000000   

      r

### Interaction Terms

In [9]:
# interactingTerms = [["relvol_nt_0", col] for col in x_cols[:-1]]
# interactingTerms

In [10]:
# interactingTerms_df = helper.get_df_with_interaction_terms(train_df, interactingTerms)

In [11]:
# interactingTerms_df.head()

In [12]:
# rosy: making my own interacting terms

# col_pairs = [
#     ['relvol_nt_0', 'rrirpnxm_nt_0'],
#     ["relvol_lst15_0", "rrirpnxm_lst15_0"],
#     ["relvol_toxhr_0", "rrirpnxm_toxhr_0"],
#     ["relvol_lsthrx15_0", "rrirpnxm_lsthrx15_0"],
# ]
# interaction_terms_train_df = helper.get_df_with_interaction_terms(train_df, col_pairs)
# interaction_terms_test_df = helper.get_df_with_interaction_terms(test_df, col_pairs)

### Transform the data if needed

#### Ordinary Least Squares

In [18]:
ols_regression_model = helper.Regression('OLS')
weights = helper.get_weights(train_df)
model_attributes = ols_regression_model.train(train_df, RESPONSE_NAME, weights)
model_attributes

array([ 3.82282182e-02,  8.60705708e-04, -5.26926641e-03, -6.02416012e-03,
        3.63656242e-05,  4.00677571e-05])

In [19]:
ols_regression_model.evaluateModel(TEST_START_DATE, TEST_END_DATE, x_cols)

Average Weighted Correlation: 0.5178389461797139
Average Mean Return: 0.0002459467385268191
Average weighted Scale Factor: 0.9629946679463024


(0.5178389461797139, 0.0002459467385268191, 0.9629946679463024)

#### OLS w/ Interacting Terms

In [15]:
ols_interacting_model = helper.Regression('OLS')
#ols_interacting_model.execute(interactingTerms_df, RESPONSE_NAME, interactingTerms_df)
# ols_interacting_model.get_metric()

#### LASSO

In [21]:
lasso_model = helper.Regression('LASSO')
weights = helper.get_weights(train_df)
lasso_model.train(train_df, RESPONSE_NAME) # bug, LASSO doesn't take in weights
lasso_model.evaluateModel(TEST_START_DATE, TEST_END_DATE, x_cols)

TypeError: cannot unpack non-iterable float object

### XGBoost



In [22]:
xgb_model = helper.Regression('XGboost')
weights = helper.get_weights(train_df)
xgb_model.train(train_df, RESPONSE_NAME, weights)
xgb_model.evaluateModel(TEST_START_DATE, TEST_END_DATE, x_cols)

/usr/local/lib/python3.10/site-packages/xgboost/core.py:726: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Average Weighted Correlation: 0.5178389460065874
Average Mean Return: 0.0002459467385268191
Average weighted Scale Factor: 0.9629945158958435


(0.5178389460065874, 0.0002459467385268191, 0.9629945)